In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d


from transformer_IDAO import TransformerIDAO
from clustering.clustering import Clustering2D
from prediction import Model2D
from validation.IDAO_validator_2d_model import IDAO_validator_2d_model
import importlib

%matplotlib inline 

In [2]:
#full_test = pd.read_csv("./data/test.csv", index_col='id')
#full_train = pd.read_csv("./data/train.csv", index_col='id')
full_test = pd.read_csv("./data/test.csv", index_col='id')
full_train = pd.read_csv("./data/train.csv", index_col='id')

transformer = TransformerIDAO()

idao_2d_splitter = IDAO_validator_2d_model(5, 5)

In [3]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [ ]:
## список из значений smape (на тесте) для каждого из sat_id
smape_per_sat_id = []
## номер фолда для вывода логов
n_folds = 0


## пока проверяю предсказания в 2D!
target_columns = ['x_pred', 'y_pred', 'Vx_pred', 'Vy_pred']#, 'Vz_pred']
true_columns = ['x', 'y', 'Vx', 'Vy']#, 'Vz']


for trn_idxs, tst_idxs in idao_2d_splitter.split(full_train):
    train_df = None
    test_df = None
    for sat_id in full_train['sat_id'].unique():
        if train_df is None:
            train_df = full_train.iloc[trn_idxs[sat_id, 0]:trn_idxs[sat_id, 1]].copy()
        else:
            train_df = pd.concat([train_df, full_train.iloc[trn_idxs[sat_id, 0]:trn_idxs[sat_id, 1]]], axis=0).copy()
        if test_df is None:
            test_df = full_train.iloc[tst_idxs[sat_id, 0]:tst_idxs[sat_id, 1]].copy()
        else:
            test_df = pd.concat([test_df, full_train.iloc[tst_idxs[sat_id, 0]:tst_idxs[sat_id, 1]]], axis=0).copy()
    
    transformer.fit_transform(train_df)

    clustering1 = Clustering2D('AgglomerativeClustering', 400)
    clusters_map = clustering1.fit_predict(transformer.train_coefs)

    model = Model2D(10)

    test_df = model.predict(train_df.copy(), test_df.copy(), clusters_map)
    
    test_df['z_pred'] = 0.0
    test_df['Vz_pred'] = 0.0
    ## так как проверяю предсказания в  2d
#     test_df = transformer.inv_transform(test_df)
    
    n_folds += 1
    print(f'fold: {n_folds}, target_col: {target_columns},\
                  smape: {1 - smape(test_df[target_columns].values, test_df[true_columns].values):.5f}')


/home/paniquex/Documents/idao_oski/transformer_IDAO.py:35: RuntimeWarning: invalid value encountered in true_divide
  a /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:36: RuntimeWarning: invalid value encountered in true_divide
  b /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:37: RuntimeWarning: invalid value encountered in true_divide
  c /= norm


COORDS HAS ENUMERATED
fold: 1, target_col: ['x_pred', 'y_pred', 'Vx_pred', 'Vy_pred'],                  smape: 0.13640


/home/paniquex/Documents/idao_oski/transformer_IDAO.py:35: RuntimeWarning: invalid value encountered in true_divide
  a /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:36: RuntimeWarning: invalid value encountered in true_divide
  b /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:37: RuntimeWarning: invalid value encountered in true_divide
  c /= norm


COORDS HAS ENUMERATED


In [ ]:
transformer.fit_transform(full_train)
print(full_train.columns)

In [ ]:
clustering1 = Clustering2D('AgglomerativeClustering', 400)
clusters_map = clustering1.fit_predict(transformer.train_coefs)

In [ ]:
model = Model2D(WIDTH=10)
new_test = model.predict(full_train.copy(), full_test.copy(), clusters_map)

In [ ]:
# new_test['z_pred'] = 0.0
# new_test['Vz_pred'] = 0.0
# new_test = transformer.inv_transform(new_test)

In [ ]:
full_test = new_test

In [ ]:
# отрисовка нужного рядя для анализа
import numpy as np
import pandas as pd
import lightgbm as lgbm
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import time
from sklearn.linear_model import LinearRegression
from tqdm import tqdm_notebook as tqdm


figure = plt.figure(figsize=(10, 5))


sat_id = 32
real_col_name = 'x_transformed'
sim_col_name = real_col_name + '_sim'
pred_col_name = real_col_name + '_pred'

fig = go.Figure()
fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
                             y=full_test[full_test.sat_id == sat_id]['x_pred'],
                             name="Estimated " + real_col_name,
                             ))

# fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
#                          y=full_test[full_test.sat_id == sat_id][sim_col_name],
#                          name="Simulation " + real_col_name,
#                          ))

# fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
#                          y=full_test[full_test.sat_id == sat_id][sim_col_name],
#                          name="Sim_test " + real_col_name,
#                          ))
fig.add_trace(go.Scatter(x=full_train[full_train.sat_id == sat_id].epoch,
                         y=full_train[full_train.sat_id == sat_id][real_col_name],
                         name="Real " + real_col_name,
                         ))

# fig.add_trace(go.Scatter(x=full_train[full_train.sat_id == sat_id].epoch,
#                          y=full_train[full_train.sat_id == sat_id][sim_col_name],
#                          name="Simulation " + real_col_name,
#                          ))

fig.update_layout(title_text='Time Series for ' + real_col_name,
                  xaxis_rangeslider_visible=True,
                  yaxis_title=real_col_name)
fig.show()

In [ ]:
full_test